In [ ]:
!pip install kaggle --upgrade
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -O ./spark-3.3.1-bin-hadoop3.tgz  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar zxvf ./spark-3.3.1-bin-hadoop3.tgz
!pip install findspark
!pip install pyspark
import findspark
import random
import pyspark
from pyspark.sql import functions as f
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/kaggle/working/spark-3.3.1-bin-hadoop3"
findspark.init()
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treatmeant').getOrCreate()

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
from pyspark.ml import Pipeline

In [ ]:
# Use Spark to read the training csv file.
data = spark.read.csv("/kaggle/working/df_maens.csv", header=True)
cols = data.columns[2:]

In [ ]:
for col_name in cols:
    data = data.withColumn(col_name,col(col_name).cast("float"))

In [ ]:
assembler = VectorAssembler(inputCols=data.columns[2:], outputCol="features",handleInvalid="keep")
model = KMeans(k=2, seed=1)
pipeline = Pipeline(stages=[assembler, model])

In [ ]:
pipeline = pipeline.fit(data)
preds = pipeline.transform(data)

In [ ]:
preds.toPandas()

In [ ]:
from sklearn.decomposition import PCA
df = preds.toPandas()
pca = PCA(n_components=2)
df_pca = pd.DataFrame(pca.fit_transform(df[cols]))
df_pca['preds'] = preds.toPandas()['prediction']
df[0] = df_pca[0]
df[1] = df_pca[1]
df['preds'] = df_pca['preds']

import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
plt.figure(figsize = (10, 7))
plt.title('Principal component analysis (PCA) means df VS Clusters', fontdict={'fontsize': 25})
sns.scatterplot(data=df, x=0, y=1, hue="preds", legend="full")